In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import roboticstoolbox as rtb

ModuleNotFoundError: No module named 'roboticstoolbox'

In [ ]:
H1 = 10
W1 = 10
L1 = 40
W2 = 5
H2 = 5
L2 = 40

In [ ]:
link1 = rtb.RevoluteDH(d=H1, a=0, alpha=0)
link2 = rtb.RevoluteDH(d=0, a=0, alpha=np.pi/2)
link3 = rtb.RevoluteDH(d=W1, a=L1, alpha=np.pi)
link4 = rtb.RevoluteDH(d=W2, a=L2, alpha=np.pi/2)
link5 = rtb.RevoluteDH(d=W2, a=0, alpha=-np.pi/2)
link6 = rtb.RevoluteDH(d=H2, a=0, alpha=-np.pi/2)

NameError: name 'rtb' is not defined

In [ ]:
robot = rtb.DHRobot([link1, link2, link3, link4, link5, link6], name='robot')
robot

NameError: name 'rtb' is not defined

In [ ]:
q = np.deg2rad([0, 0, 0, 0, 0, 0])